In [1]:
import numpy as np

In [2]:
def reconstruct(pmax, phi):
    """
    Ricostruisce tutti i most probable, per ora fa schifo ma sembra funzionare, da ricontrollare
    """
    reconstruction = np.empty(len(phi) + 1)

    curr = np.argmax(pmax[-1])
    reconstruction[-1] = curr

    for i in range(len(phi) - 1, -1, -1):
        curr = int(phi[i, curr])
        reconstruction[i] = curr

    return reconstruction

### Experiement with matrix like the one we have (last row of spaces has 1)


In [3]:
A = np.array([[0.1, 0.8, 0.1], [0.8, 0.1, 0.1], [0.5, 0.5, 0]])

B = np.array([[0.8, 0.2, 0], [0.2, 0.8, 0], [0, 0, 1]])

observed = np.array([0, 2, 1])

In [4]:
def compute_f(A, B, observed):
    """
    Careful when handling
    - f0 is the first message (from first factor to node, it is just a vector)
    - f contains all other factors evaluated
    """
    pi = A[-1]
    n_nodes = len(observed)
    n_states = A.shape[0]
    f = np.zeros((n_nodes - 1, n_states, n_states))

    tmp = np.zeros((n_states, 1))
    for k in range(n_states):
        tmp[k] = pi[k] * B[k, observed[0]]

    f0 = tmp

    for i in range(1, n_nodes):
        tmp = np.zeros((n_states, n_states))

        for j in range(n_states):  # over z1
            for k in range(n_states):  # over z2
                tmp[j, k] = A[j, k] * B[k, observed[i]]

        f[i - 1] = tmp

    return f0, f

In [5]:
f0, f = compute_f(A, B, observed)
print("f0:\n", f0, "\nf:\n", f)

f0:
 [[0.4]
 [0.1]
 [0. ]] 
f:
 [[[0.   0.   0.1 ]
  [0.   0.   0.1 ]
  [0.   0.   0.  ]]

 [[0.02 0.64 0.  ]
  [0.16 0.08 0.  ]
  [0.1  0.4  0.  ]]]


In [6]:
def Viterbi(f0, f):
    n_nodes = f.shape[0] + 1
    n_states = f.shape[1]

    pmax = np.zeros((n_nodes, n_states))  # Need one for every node
    phi = np.zeros(
        (n_nodes - 1, n_states)
    )  # Need one for every node other than the first one (no need to reconstruct it)

    pmax[0] = f0.flatten()

    for i in range(1, n_nodes):
        tmp = ((f[i - 1]).T * pmax[i - 1]).T

        pmax[i] = np.max(tmp, axis=0)  # by column

        phi[i - 1] = np.argmax(
            tmp, axis=0
        )  # i-1 cause this contains the reconstruction about the (i-1)th element

    return pmax, phi

In [7]:
pmax, phi = Viterbi(f0, f)
print("pmax:\n", pmax, "\nphi:\n", phi)

print("\nReconstruction using Viterbi:\n", reconstruct(pmax, phi))

pmax:
 [[0.4   0.1   0.   ]
 [0.    0.    0.04 ]
 [0.004 0.016 0.   ]] 
phi:
 [[0. 0. 0.]
 [2. 2. 0.]]

Reconstruction using Viterbi:
 [0. 2. 1.]


In [8]:
observed = np.array([0, 2, 1, 0, 2, 0, 1, 0, 2, 1, 0, 1, 0])  # a _ b a
f0, f = compute_f(A, B, observed)
pmax, phi = Viterbi(f0, f)
print("pmax:\n", pmax, "\nphi:\n", phi)

print("\nReconstruction using Viterbi:\n", reconstruct(pmax, phi))

pmax:
 [[4.00000000e-01 1.00000000e-01 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 4.00000000e-02]
 [4.00000000e-03 1.60000000e-02 0.00000000e+00]
 [1.02400000e-02 6.40000000e-04 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 1.02400000e-03]
 [4.09600000e-04 1.02400000e-04 0.00000000e+00]
 [1.63840000e-05 2.62144000e-04 0.00000000e+00]
 [1.67772160e-04 5.24288000e-06 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 1.67772160e-05]
 [1.67772160e-06 6.71088640e-06 0.00000000e+00]
 [4.29496730e-06 2.68435456e-07 0.00000000e+00]
 [8.58993459e-08 2.74877907e-06 0.00000000e+00]
 [1.75921860e-06 5.49755814e-08 0.00000000e+00]] 
phi:
 [[0. 0. 0.]
 [2. 2. 0.]
 [1. 0. 0.]
 [0. 0. 0.]
 [2. 2. 0.]
 [1. 0. 0.]
 [1. 1. 0.]
 [0. 0. 0.]
 [2. 2. 0.]
 [1. 0. 0.]
 [0. 0. 0.]
 [1. 1. 0.]]

Reconstruction using Viterbi:
 [0. 2. 1. 0. 2. 0. 1. 0. 2. 1. 0. 1. 0.]


# Log computations

Now we need to find a way for this to hold even when working with logs.
I suspect that replacing with the minimum to account for the 0 is too big, we need something smaller.


In [9]:
A = np.array([[0.1, 0.8, 0.1], [0.8, 0.1, 0.1], [0.5, 0.5, 0]])

B = np.array([[0.8, 0.2, 0], [0.2, 0.8, 0], [0, 0, 1]])

observed = np.array([0, 2, 1])

In [10]:
def compute_f_log(A, B, observed):
    """
    Even though it contains the -inf, this is fine, as that means we should not reconstruct using those (?)
    """
    pi = A[-1]
    n_nodes = len(observed)
    n_states = A.shape[0]
    f = np.zeros((n_nodes - 1, n_states, n_states))

    tmp = np.zeros((n_states, 1))
    for k in range(n_states):
        tmp[k] = np.log(pi[k]) + np.log(B[k, observed[0]])

    f0 = tmp

    for i in range(1, n_nodes):
        tmp = np.zeros((n_states, n_states))

        for j in range(n_states):  # over z1
            for k in range(n_states):  # over z2
                tmp[j, k] = np.log(A[j, k]) + np.log(B[k, observed[i]])

        f[i - 1] = tmp

    return f0, f

In [11]:
def Viterbi_log(f0, f):
    n_nodes = f.shape[0] + 1
    n_states = f.shape[1]

    pmax = np.zeros((n_nodes, n_states))  # Need one for every node
    phi = np.zeros(
        (n_nodes - 1, n_states)
    )  # Need one for every node other than the first one (no need to reconstruct it)

    pmax[0] = f0.flatten()

    for i in range(1, n_nodes):
        tmp = ((f[i - 1]).T + pmax[i - 1]).T

        pmax[i] = np.max(tmp, axis=0)  # by column

        phi[i - 1] = np.argmax(
            tmp, axis=0
        )  # i-1 cause this contains the reconstruction about the (i-1)th element

    return pmax, phi

In [12]:
f0, f = compute_f_log(A, B, observed)
print("f0:\n", f0, "\nf:\n", f)


pmax, phi = Viterbi_log(f0, f)
print("\n\npmax:\n", pmax, "\nphi:\n", phi)

print("\n\n reconstruction:", reconstruct(pmax, phi))

f0:
 [[-0.91629073]
 [-2.30258509]
 [       -inf]] 
f:
 [[[       -inf        -inf -2.30258509]
  [       -inf        -inf -2.30258509]
  [       -inf        -inf        -inf]]

 [[-3.91202301 -0.4462871         -inf]
  [-1.83258146 -2.52572864        -inf]
  [-2.30258509 -0.91629073        -inf]]]


pmax:
 [[-0.91629073 -2.30258509        -inf]
 [       -inf        -inf -3.21887582]
 [-5.52146092 -4.13516656        -inf]] 
phi:
 [[0. 0. 0.]
 [2. 2. 0.]]


 reconstruction: [0. 2. 1.]


/var/folders/tv/ysw0jsvs2t331mtk7hdh35kc0000gn/T/ipykernel_65691/2882175090.py:12: RuntimeWarning: divide by zero encountered in log
  tmp[k] = np.log(pi[k]) + np.log(B[k, observed[0]])
/var/folders/tv/ysw0jsvs2t331mtk7hdh35kc0000gn/T/ipykernel_65691/2882175090.py:21: RuntimeWarning: divide by zero encountered in log
  tmp[j, k] = np.log(A[j, k]) + np.log(B[k, observed[i]])


In [13]:
observed = np.array([0, 2, 1, 2, 1, 0, 1])  # a = 0; b = 1; _ = 2
f0, f = compute_f_log(A, B, observed)
print("f0:\n", f0, "\nf:\n", f)


pmax, phi = Viterbi_log(f0, f)
print("\n\npmax:\n", pmax, "\nphi:\n", phi)

print("\n\n reconstruction:", reconstruct(pmax, phi))

f0:
 [[-0.91629073]
 [-2.30258509]
 [       -inf]] 
f:
 [[[       -inf        -inf -2.30258509]
  [       -inf        -inf -2.30258509]
  [       -inf        -inf        -inf]]

 [[-3.91202301 -0.4462871         -inf]
  [-1.83258146 -2.52572864        -inf]
  [-2.30258509 -0.91629073        -inf]]

 [[       -inf        -inf -2.30258509]
  [       -inf        -inf -2.30258509]
  [       -inf        -inf        -inf]]

 [[-3.91202301 -0.4462871         -inf]
  [-1.83258146 -2.52572864        -inf]
  [-2.30258509 -0.91629073        -inf]]

 [[-2.52572864 -1.83258146        -inf]
  [-0.4462871  -3.91202301        -inf]
  [-0.91629073 -2.30258509        -inf]]

 [[-3.91202301 -0.4462871         -inf]
  [-1.83258146 -2.52572864        -inf]
  [-2.30258509 -0.91629073        -inf]]]


pmax:
 [[ -0.91629073  -2.30258509         -inf]
 [        -inf         -inf  -3.21887582]
 [ -5.52146092  -4.13516656         -inf]
 [        -inf         -inf  -6.43775165]
 [ -8.74033674  -7.35404238        

/var/folders/tv/ysw0jsvs2t331mtk7hdh35kc0000gn/T/ipykernel_65691/2882175090.py:12: RuntimeWarning: divide by zero encountered in log
  tmp[k] = np.log(pi[k]) + np.log(B[k, observed[0]])
/var/folders/tv/ysw0jsvs2t331mtk7hdh35kc0000gn/T/ipykernel_65691/2882175090.py:21: RuntimeWarning: divide by zero encountered in log
  tmp[j, k] = np.log(A[j, k]) + np.log(B[k, observed[i]])


### Try using our words example

Now using the matrix and our phrase we try and see if it works


In [14]:
from src.CipherUtils import CipherGenerator
from src.CipherUtils import TextEncoder
from src.ProbabilityMatrix import ProbabilityMatrix
from src.CipherUtils import TextPreProcessor

from src.HMM_utils import map_alphabet_to_numbers, string_to_numbers
from src.HMM_utils import find_mapping, numbers_to_string, invert_mapping

from src.HMM_functions import Baum_Welch

In [15]:
# hidden_sequence = "people of western europe a landing was made this morning on the coast of france by troops kangaroo jokes quasi vile xilophone zenit "
hidden_sequence = "in germany it seems to be pretty much automatic pretty much all the time in france and spain it all just depends presumably on social subtleties that you have to be french or spanish to understand in italy why would you even just bother when and how much to tip is a question that has been vexing visitors to europe for as long as people have been travelling around the continent outside their own country it seems even europeans don t know the answer according to new polling by yougov in six eu countries britain and the us where as most visitors know but may be reluctant to acknowledge gratuities may make up more than half your waitperson s income europeans are deeply divided on tipping in restaurants for example of respondents in germany told the pollster they typically tipped almost the same as the us in the uk where an optional service charge of about is usually included said they left a gratuity the figure in spain where service is often included in restaurant bills but diners can leave optional tips was while in france where every price on a restaurant menu already includes for service of people said they generally tipped on top even in sweden where tips are generally not expected the figure was but only of italians said they would typically leave a gratuity after a meal out with a rather greater proportion admitting they never left a cent a startling of respondents in the us however and of germans by far the most in europe confessed they would tip sometimes or often even if the service was terrible indicating that for some tipping is not about quality of service at all the findings of the survey will come as a surprise in germany a country that does not generally think of itself as a nation of happy distributors"
# hidden_sequence = "hello banana xilophone key queue zebra cock pussy tits dandy fart though jolly world mum "
hidden_sequence = "I pinched it out of the skivvy room, Buck Mulligan said. It does her all right. The aunt always keeps plainlooking servants for Malachi. Lead him not into temptation. And her name is Ursula. Laughing again, he brought the mirror away from Stephen peering eyes. The rage of Caliban at not seeing his face in a mirror, he said. If Wilde were only alive to see you Cracked lookingglass of a servant! Tell that to the oxy chap downstairs and touch him for a guinea. He  stinking with money and thinks you are not a gentleman. His old fellow made his tin by selling jalap to Zulus or some bloody swindle or other. God, Kinch, if you and I could only work together we might do something for the island. Hellenise it.  Young shouts of moneyed voices in Clive Kempthorpe  rooms. Palefaces: they hold their ribs with laughter, one clasping another. O, I shall expire! Break the news to her gently, Aubrey! I shall die! With slit ribbons of his shirt whipping the air he hops and hobbles round the table, with trousers down at heels, chased by Ades of Magdalen with the tailor shears. A scared face gilded with marmalade. I don want to be debagged! Don t you play the giddy ox with me!"

preprocessor = TextPreProcessor()
hidden_sequence = preprocessor.lower(text=hidden_sequence)
hidden_sequence = preprocessor.remove_unknown_chars(
    text=hidden_sequence, unknown_chars=preprocessor.unknown_chars(hidden_sequence)
)
hidden_sequence = preprocessor.remove_additional_spaces(text=hidden_sequence)


cipher_generator = CipherGenerator()
cipher = cipher_generator.generate_cipher()
encoder = TextEncoder()
observed_sequence = encoder.encode_text(hidden_sequence, cipher=cipher)

# Convert to numeric
hidden_ = string_to_numbers(hidden_sequence, mapping=map_alphabet_to_numbers())
observed_ = string_to_numbers(observed_sequence, mapping=map_alphabet_to_numbers())

print(hidden_sequence)
print(observed_sequence)

i pinched it out of the skivvy room buck mulligan said it does her all right the aunt always keeps plainlooking servants for malachi lead him not into temptation and her name is ursula laughing again he brought the mirror away from stephen peering eyes the rage of caliban at not seeing his face in a mirror he said if wilde were only alive to see you cracked lookingglass of a servant tell that to the oxy chap downstairs and touch him for a guinea he stinking with money and thinks you are not a gentleman his old fellow made his tin by selling jalap to zulus or some bloody swindle or other god kinch if you and i could only work together we might do something for the island hellenise it young shouts of moneyed voices in clive kempthorpe rooms palefaces they hold their ribs with laughter one clasping another o i shall expire break the news to her gently aubrey i shall die with slit ribbons of his shirt whipping the air he hops and hobbles round the table with trousers down at heels chased b

In [16]:
# List of text file paths to build our corpus (where we learn the transitions probs)
file_paths = [
    "texts/moby_dick.txt",
    "texts/shakespeare.txt",
    "texts/james-joyce-a-portrait-of-the-artist-as-a-young-man.txt",
    "texts/james-joyce-dubliners.txt",
    "texts/james-joyce-ulysses.txt",
]

texts = []
for file_path in file_paths:
    with open(file_path, "r") as file:
        texts.append(file.read())

corpus = "".join(texts)
alphabet = list("abcdefghijklmnopqrstuvwxyz ")

preprocessor = TextPreProcessor()
corpus = preprocessor.lower(text=corpus)
corpus = preprocessor.remove_unknown_chars(
    text=corpus, unknown_chars=preprocessor.unknown_chars(corpus)
)
corpus = preprocessor.remove_additional_spaces(text=corpus)

# compute probabilities
p = ProbabilityMatrix(corpus)
p.compute_probability_matrix()
p.compute_normalized_matrix()
# p.compute_probability_table()

In [17]:
B_start = np.zeros((27, 27)) + 1 / 26
B_start[:, -1] = np.zeros(27)
B_start[-1, :] = np.zeros(27)
B_start[-1, -1] = 1

emission = Baum_Welch(
    A=p.normalized_matrix,
    B_start=B_start,
    pi=p.normalized_matrix[-1, :],
    observed=observed_,
    maxIter=50,
)
mapping = find_mapping(emission.argmax(axis=1))
normalized_emission_reconstruction = numbers_to_string(
    observed_sequence, invert_mapping(mapping)
)

In [18]:
B_start = np.zeros((27, 27)) + 1 / 26
B_start[:, -1] = np.zeros(27)
B_start[-1, :] = np.zeros(27)
B_start[-1, -1] = 1

emission_non_norm = Baum_Welch(
    A=p.probability_matrix,
    B_start=B_start,
    pi=p.probability_matrix[-1, :],
    observed=observed_,
    maxIter=50,
)
mapping_non_norm = find_mapping(emission_non_norm.argmax(axis=1))
non_normalized_emission_reconstruction = numbers_to_string(
    observed_sequence, invert_mapping(mapping_non_norm)
)

In [19]:
f0, f = compute_f_log(A=p.normalized_matrix, B=emission, observed=observed_)
pmax, phi = Viterbi_log(f0, f)
reconstruction = reconstruct(pmax, phi)
reconstruction = reconstruction.astype(int)

/var/folders/tv/ysw0jsvs2t331mtk7hdh35kc0000gn/T/ipykernel_65691/2882175090.py:12: RuntimeWarning: divide by zero encountered in log
  tmp[k] = np.log(pi[k]) + np.log(B[k, observed[0]])
/var/folders/tv/ysw0jsvs2t331mtk7hdh35kc0000gn/T/ipykernel_65691/2882175090.py:21: RuntimeWarning: divide by zero encountered in log
  tmp[j, k] = np.log(A[j, k]) + np.log(B[k, observed[i]])


In [20]:
f0, f = compute_f_log(A=p.probability_matrix, B=emission_non_norm, observed=observed_)
pmax, phi = Viterbi_log(f0, f)
reconstruction_non_norm = reconstruct(pmax, phi)
reconstruction_non_norm = reconstruction_non_norm.astype(int)

/var/folders/tv/ysw0jsvs2t331mtk7hdh35kc0000gn/T/ipykernel_65691/2882175090.py:12: RuntimeWarning: divide by zero encountered in log
  tmp[k] = np.log(pi[k]) + np.log(B[k, observed[0]])
/var/folders/tv/ysw0jsvs2t331mtk7hdh35kc0000gn/T/ipykernel_65691/2882175090.py:21: RuntimeWarning: divide by zero encountered in log
  tmp[j, k] = np.log(A[j, k]) + np.log(B[k, observed[i]])


In [21]:
import numpy as np


def convert_numbers_to_letters(numbers):
    letters = []
    for number in numbers:
        if number == 26:
            letters.append(" ")
        else:
            letter = chr(number + ord("a"))
            letters.append(letter)
    return "".join(letters)


from src.HMM_utils import numbers_to_string, map_alphabet_to_numbers, invert_mapping


def convert_numbers_to_letters(numbers):
    return numbers_to_string(numbers, invert_mapping(map_alphabet_to_numbers()))

In [22]:
normalized_reconstruction = convert_numbers_to_letters(reconstruction)
non_normalized_reconstruction = convert_numbers_to_letters(reconstruction_non_norm)

print(
    "Non-normalized emission reconstruction:\n", non_normalized_emission_reconstruction
)
print("Non-normalized reconstruction:\n", non_normalized_reconstruction)

print("\n")

print("Normalized emission reconstruction:\n", normalized_emission_reconstruction)
print("Normalized reconstruction:\n", normalized_reconstruction)

Non-normalized emission reconstruction:
 i einihed it out of the skizzy room xuik mulligan said it does her all right the aunt always keees elainlooking serzants for malaihi lead him not into temetation and her name is ursula laughing again he xrought the mirror away from steehen eeering eyes the rage of ialixan at not seeing his faie in a mirror he said if wilde were only alize to see you iraiked lookingglass of a serzant tell that to the ody ihae downstairs and touih him for a guinea he stinking with money and thinks you are not a gentleman his old fellow made his tin xy selling aalae to qulus or some xloody swindle or other god kinih if you and i iould only work together we might do something for the island hellenise it young shouts of moneyed zoiies in ilize kemethoree rooms ealefaies they hold their rixs with laughter one ilaseing another o i shall edeire xreak the news to her gently auxrey i shall die with slit rixxons of his shirt whieeing the air he hoes and hoxxles round the t

In [23]:
print(normalized_reconstruction)
print(hidden_sequence)

i pinched it out of the skizzy foom buck mulligan said it dous her all right the aunt always keess plainlooking sercants for malachi lead him not into temptation and her name is ursula langhing again he brought the mirror away from steshen peering eres the fave of caliman at not sexing his fave in a mirror he said if wilde were only alive to see you cracked lookingglass of a sercant tell that to the omy chas downstairs and touch him for a buinea he stinking with money and thinks you are not a gentheman his old fellow mave his tin by selling walas to qulus or some bloody swindle or other god kinch if you and i could only work tovether we might do something for the island hellenise it young shouts of monered coices in clive kempthorpe fooms palefaves they hold their rims with langhter one clasping another o i shall expire break the ners to her gently auprey i shall die with blit rimbons of his shirt whipping the air he hous and hombles found the table with trousers down at heels chased b

In [24]:
print(
    "normalized emission accuracy:",
    np.mean(
        np.array(list(normalized_emission_reconstruction))
        == np.array(list(hidden_sequence))
    ),
)
print(
    "normalized reconstruction accuracy:",
    np.mean(
        np.array(list(normalized_reconstruction)) == np.array(list(hidden_sequence))
    ),
)

print(
    "NON normalized emission accuracy:",
    np.mean(
        np.array(list(non_normalized_emission_reconstruction))
        == np.array(list(hidden_sequence))
    ),
)
print(
    "NON normalized reconstruction accuracy:",
    np.mean(
        np.array(list(non_normalized_reconstruction)) == np.array(list(hidden_sequence))
    ),
)

normalized emission accuracy: 0.9578577699736611
normalized reconstruction accuracy: 0.9604916593503073
NON normalized emission accuracy: 0.9446883230904302
NON normalized reconstruction accuracy: 0.8946444249341527
